# User API

In [1]:
%config InlineBackend.print_figure_kwargs = {'bbox_inches': 'tight', 'dpi': 110}
%load_ext autoreload
%autoreload 2
import logging, warnings
logging.getLogger("pymc").setLevel(logging.FATAL)
warnings.filterwarnings("ignore")

## PyMC

The [Example](example.html) page introduces how to use *muse-inference* for a problem defined with PyMC. Here we consider a more complex problem to highlight additional features. In particular:

* We can estimate any number of parameters with any shapes. Here we have a 2-dimensional array $\mu$ and a scalar $\theta$. Note that by default, *muse-inference* considers any variables which do not depend on others as "parameters" (i.e. the "leaves" of the probabilistic graph). However, the algorithm is not limited to such parameters, and any choice can be selected by providing a list of `params` to the `PyMCMuseProblem` constructor.

* We can work with distributions with limited domain support. For example, below we use the $\rm Beta$ distribution with support on $(0,1)$ and the $\rm LogNormal$ distribution with support on $(0,\infty)$. All necessary transformations are handled internally.

* The data and latent space can include any number of variables, with any shapes. Below we demonstrate an $x$ and $z$ which are 2-dimensional arrays. 

First, load the relevant packages:

In [2]:
%pylab inline
import pymc as pm
from muse_inference.pymc import PyMCMuseProblem

Populating the interactive namespace from numpy and matplotlib


Then define the problem,

In [3]:
def gen_funnel(x=None, θ=None, μ=None, rng=None):
    with pm.Model(rng_seeder=rng) as model:
        μ = pm.Beta("μ", 2, 5, size=2) if μ is None else μ
        θ = pm.Normal("θ", 0, 3) if θ is None else θ
        z = pm.LogNormal("z", μ, np.exp(θ/2), size=(100, 2))
        x = pm.Normal("x", z, 1, observed=x)
    return model

generate the model and some data, given some chosen true values of parameters,

In [4]:
params_true = dict(μ=[0.3, 0.7], θ=1)
x_obs = pm.sample_prior_predictive(1, gen_funnel(rng=RandomState(0), **params_true)).prior.x[0,0]
model = gen_funnel(x=x_obs)
prob = PyMCMuseProblem(model)

and finally, run MUSE:

In [5]:
params_start = dict(μ=[0.5, 0.5], θ=0)
result = prob.solve(params_start, progress=True)

MUSE:   0%|          | 0/5050 [00:00<?, ?it/s]

MUSE:   0%|          | 9/5050 [00:00<00:57, 87.38it/s]

MUSE:   0%|          | 18/5050 [00:00<00:58, 86.47it/s]

MUSE:   1%|          | 27/5050 [00:00<00:59, 84.71it/s]

MUSE:   1%|          | 36/5050 [00:00<01:00, 83.26it/s]

...

get_H:  77%|███████▋  | 54/70 [00:01<00:00, 43.61it/s]

get_H:  84%|████████▍ | 59/70 [00:01<00:00, 43.51it/s]

get_H:  93%|█████████▎| 65/70 [00:01<00:00, 45.10it/s]

get_H: 100%|██████████| 70/70 [00:01<00:00, 43.13it/s]

get_H: 100%|██████████| 70/70 [00:01<00:00, 44.75it/s]

When there are multiple parameters, the starting guess should be specified as as a dictionary, as above.

The solution is returned as a 1-dimensional vector of all parameters concatenated in the order they appear in the model:

In [6]:
result.θ, result.Σ

(array([0.32164014, 0.52036869, 0.81806896]),
 array([[ 0.02828987,  0.00559278, -0.0216605 ],
        [ 0.00559278,  0.01515474, -0.00759072],
        [-0.0216605 , -0.00759072,  0.02698495]]))

## Jax

We can also use [Jax](https://jax.readthedocs.io/) to define the problem. In this case we will write out function to generate forward samples and to compute the posterior, and Jax will provide necessary gradients for free. To use Jax, load the necessary packages:

In [7]:
from functools import partial
import jax
import jax.numpy as jnp
from muse_inference.jax import JittableJaxMuseProblem, JaxMuseProblem
from muse_inference import XZSample

Let's implement the noisy funnel problem from the [Example](example.html) page. To do so, extend either `JaxMuseProblem`, or, if your code is able to be JIT compiled by Jax, extend `JittableJaxMuseProblem` and decorate the functions with `jax.jit`:

In [8]:
class JaxFunnelMuseProblem(JittableJaxMuseProblem):

    def __init__(self, N):
        super().__init__()
        self.N = N

    @partial(jax.jit, static_argnums=0)
    def sample_x_z(self, key, θ):
        keys = jax.random.split(key, 2)
        z = jax.random.normal(keys[0], (self.N,)) * jnp.exp(θ/2)
        x = z + jax.random.normal(keys[1], (self.N,))
        return XZSample(x, z)

    @partial(jax.jit, static_argnums=0)
    def logLike(self, x, z, θ):
        return -(jnp.sum((x - z)**2) + jnp.sum(z**2) / jnp.exp(θ) + 512*θ) / 2

    @partial(jax.jit, static_argnums=0)
    def logPrior(self, θ):
        return -θ**2 / (2*3**2)

Now generate some simulated data, which we set into `prob.x`. Note also the use of `PRNGKey` (rather than `RandomState` for PyMC/Numpy) for random number generation. 

In [9]:
prob = JaxFunnelMuseProblem(10000)
key = jax.random.PRNGKey(0)
(x, z) = prob.sample_x_z(key, jnp.array([1.]))
prob.x = x

And finally, run MUSE:

In [10]:
result = prob.solve(θ_start=0., rng=jax.random.PRNGKey(1), progress=True)

MUSE:   0%|          | 0/5050 [00:00<?, ?it/s]

MUSE:   0%|          | 1/5050 [00:02<2:55:13,  2.08s/it]

MUSE:   2%|▏         | 102/5050 [00:05<03:58, 20.78it/s]

MUSE:   4%|▍         | 203/5050 [00:05<01:48, 44.47it/s]

MUSE:   6%|▌         | 304/5050 [00:06<01:01, 77.18it/s]

...

MUSE: 100%|██████████| 5050/5050 [00:07<00:00, 697.00it/s] 

get_H:   0%|          | 0/30 [00:00<?, ?it/s]

get_H:   3%|▎         | 1/30 [00:02<01:07,  2.33s/it]

get_H:  70%|███████   | 21/30 [00:02<00:00, 11.94it/s]

get_H: 100%|██████████| 30/30 [00:02<00:00, 11.91it/s]

Note that the solution here is obtained around 10X faster that the PyMC version of this in the [Example](example.html) page. The Jax interface has much lower overhead, which will be noticeable for very fast posteriors like the one above. 

One powerful aspect of using Jax is that the parameters, `θ`, and latent space, `z`, can be any [pytree](https://jax.readthedocs.io/en/latest/pytrees.html), ie tuples, dictionaries, nested combinations of them, etc... (there is no requirement on the data format of the `x` variable). To demonstrate, consider a problem which is just two copies of the noisy funnel problem:

In [11]:
class JaxPyTreeFunnelMuseProblem(JittableJaxMuseProblem):

    def __init__(self, N):
        super().__init__()
        self.N = N

    @partial(jax.jit, static_argnums=0)
    def sample_x_z(self, key, θ):
        (θ1, θ2) = (θ["θ1"], θ["θ2"])
        keys = jax.random.split(key, 4)
        z1 = jax.random.normal(keys[0], (self.N,)) * jnp.exp(θ1/2)
        z2 = jax.random.normal(keys[1], (self.N,)) * jnp.exp(θ2/2)        
        x1 = z1 + jax.random.normal(keys[2], (self.N,))
        x2 = z2 + jax.random.normal(keys[3], (self.N,))        
        return XZSample(x={"x1":x1, "x2":x2}, z={"z1":z1, "z2":z2})

    @partial(jax.jit, static_argnums=0)
    def logLike(self, x, z, θ):
        return (
            -(jnp.sum((x["x1"] - z["z1"])**2) + jnp.sum(z["z1"]**2) / jnp.exp(θ["θ1"]) + 512*θ["θ1"]) / 2
            -(jnp.sum((x["x2"] - z["z2"])**2) + jnp.sum(z["z2"]**2) / jnp.exp(θ["θ2"]) + 512*θ["θ2"]) / 2
        )

    @partial(jax.jit, static_argnums=0)
    def logPrior(self, θ):
        return - θ["θ1"]**2 / (2*3**2) - θ["θ2"]**2 / (2*3**2)

Here, `x`, `θ`, and `z` are all dictionaries. We generate the problem as usual, passing in parameters as dictionaries,

In [12]:
θ_true = dict(θ1=-1., θ2=2.)
θ_start = dict(θ1=0., θ2=0.)

In [13]:
prob = JaxPyTreeFunnelMuseProblem(10000)
key = jax.random.PRNGKey(0)
(x, z) = prob.sample_x_z(key, θ_true)
prob.x = x

and run MUSE:

In [14]:
result = prob.solve(θ_start=θ_start, rng=jax.random.PRNGKey(0), progress=True)

MUSE:   0%|          | 0/5050 [00:00<?, ?it/s]

MUSE:   0%|          | 1/5050 [00:02<3:44:45,  2.67s/it]

MUSE:   2%|▏         | 102/5050 [00:07<05:05, 16.19it/s]

MUSE:   3%|▎         | 151/5050 [00:07<03:03, 26.72it/s]

MUSE:   4%|▍         | 199/5050 [00:07<01:59, 40.66it/s]

...

get_H:   0%|          | 0/50 [00:00<?, ?it/s]

get_H:   2%|▏         | 1/50 [00:02<02:09,  2.63s/it]

get_H:  32%|███▏      | 16/50 [00:02<00:04,  8.07it/s]

get_H:  78%|███████▊  | 39/50 [00:02<00:00, 23.22it/s]

get_H: 100%|██████████| 50/50 [00:02<00:00, 16.97it/s]

The result is returned as a dictionary:

In [15]:
result.θ

{'θ1': DeviceArray(-1.0030082, dtype=float32),
 'θ2': DeviceArray(2.0271263, dtype=float32)}

and the covariance as a matrix:

In [16]:
result.Σ

array([[ 3.3970999e-03, -3.8145434e-05],
       [-3.8145430e-05,  2.4442433e-04]], dtype=float32)